In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [3]:
import os.path
import tensorflow as tf
import numpy as np
from keras.layers import Convolution2D, Lambda
import matplotlib.pyplot as plt
from collections import deque
import random
from gym.wrappers import Monitor
from gym import logger as gymlogger
import tensorflow as tf
import numpy as np
import random
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
import keras
from keras.callbacks import TensorBoard
import os
import time
from keras.engine.topology import Layer
import shutil
from keras import optimizers,initializers
import cv2
from keras.layers import Input, Dense, Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.models import load_model,Model
from collections import deque 
import gym
from datetime import datetime
from keras.models import clone_model
from skimage.color import rgb2gray
from skimage.transform import resize
import time
from keras import losses
from keras import backend as K

Using TensorFlow backend.


In [4]:
!apt-get install cmake > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install gym[atari] 
!pip install h5py pyyaml
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install ez_setup > /dev/null 2>&1
!unzip ngrok-stable-linux-amd64.zip
!apt-get update > /dev/null 2>&1

     |████████████████████████████████| 583kB 4.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 46.0.0
    Uninstalling setuptools-46.0.0:
      Successfully uninstalled setuptools-46.0.0


--2020-04-06 11:38:09--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.70.189.149, 52.3.53.111, 52.5.68.140, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.70.189.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.1MB/s    in 0.7s    

2020-04-06 11:38:10 (18.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
def show_video():
  mlist = glob.glob('video/*.mp4')
  if len(mlist) > 0:
    mp4 = mlist[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Sorry!!!No Video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
def down_sample(gray):
    return(gray[25::2, ::2])
def dense_to_one_hot(data, depth=10):
    return(np.arange(depth) == np.array(data)[:, None]).astype(np.bool)

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
EXPLORATION_STEPS=500000
NUM_ACTIONS=4
BATCH_SIZE=32
STATE_LENGTH=4
SAVE_INTERVAL=200
MEMORY_LOAD=100000
EPSILON_INITIAL=1
EPSILON_FINAL=0.05

def rgb2gray(im):
    return(cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)).astype(np.uint8)

LEARNING_RATE_END=5e-5
TRAIN_INTERVAL=4
GAMMA=0.99
TAU=0.001
LEARNING_RATE_START=1e-4
SHAPE=(None,93,80,STATE_LENGTH)
UPDATE_TARGET_NETWORK_INTERVAL=40

In [0]:
class Q_Agent:
    def __init__(self,root):
        self.batch=deque()
        self.root=root
        self.model_target=Q_Agent.tn_build()
        self.q_out,self.model_train=Q_Agent.net_bt()
        self.epsilon, self.episode, self.learning_rate, self.decay_step = self.res()
        self.opt=optimizers.adam(lr=self.learning_rate)
        self.model_train.compile(optimizer=self.opt,loss=[Q_Agent.huber])

    def net_bt():
        mask=Input(shape=(NUM_ACTIONS,), dtype='float32')
        X=Input(shape=SHAPE[1:], dtype='float32')
        model,q_out=Q_Agent.infer(X)
        q_=Lambda(lambda x: K.reshape(K.sum(x * mask, axis=1), (-1, 1)), output_shape=(1,))(q_out)
        return(K.function([X], [q_out]), Model(inputs=[X, mask], outputs=q_))

    #Store Transactions
    def fbat(self, data):
        if(MEMORY_LOAD<=len(self.batch)):
            self.batch.popleft()
        self.batch.append(data)
        return(self.batch)

    #Model Save
    def m_save(self):
        self.model_target.save_weights(os.path.join('/content/gdrive/My Drive/Colab Notebooks/', 'model_duel_target.h5'))
        np.save(os.path.join('/content/gdrive/My Drive/Colab Notebooks/', 'params_duel'), [self.episode, self.decay_step, self.epsilon, self.learning_rate])
        self.model_train.save_weights(os.path.join('/content/gdrive/My Drive/Colab Notebooks/', 'model_duel_train.h5'))

    #Huber Loss
    def huber(x,y):
        loss=K.mean(0.5*K.square(K.clip((K.abs(x-y)),0.0,1.0))+((K.abs(x-y))-(K.clip((K.abs(x - y)),0.0,1.0))), axis=-1)
        return(loss)

    #Reward Calculation
    def sbat(self):
        q_batch, state_now, mask_b, nxt_sta, rew, done =\
            map(lambda x: np.array(list(x)), zip(*random.sample(self.batch, 32)))
        a=nxt_sta
        nxt_sta=np.transpose(a,axes=[0, 2, 3, 1])
        b=state_now
        state_now=np.transpose(b,axes=[0, 2, 3, 1])
        q_next=self.model_target.predict(nxt_sta)
        mask_b=dense_to_one_hot(mask_b, 4)
        q_batch[mask_b]=np.array(rew) + (GAMMA*np.array(done)*np.max(q_next,axis=1))
        return(q_batch,state_now,mask_b)

    def predict(self, state):
        q=self.q_out([state])
        q=np.array(q).flatten()
        return(np.argmax(q),q)

    #Convolution Layer
    def infer(X, trainable=True, init=initializers.truncated_normal(stddev=0.01)):
        bi=initializers.constant(0)
        wei=init
        #Three Convolution Layers
        nor=Lambda(lambda x: x / 255., output_shape=K.int_shape(X)[1:])(X)
        #Layer1
        convo1=Convolution2D(32, (8, 8),strides=(4, 4),kernel_initializer=wei,use_bias=False,padding='same',activation='relu')(nor)
        #Layer2
        convo2=Convolution2D(64, (4, 4),strides=(2, 2),kernel_initializer=wei,use_bias=False,padding='same',activation='relu')(convo1)
        #Layer3
        convo3=Convolution2D(64, (3, 3),strides=(1, 1),kernel_initializer=wei,use_bias=False,padding='same',activation='relu')(convo2)
        h_flat=Flatten()(convo3)
        fc_advantage = Dense(512, use_bias=False, kernel_initializer=wei, activation='relu')(h_flat)
        advantage = Dense(NUM_ACTIONS, kernel_initializer=wei, use_bias=False, bias_initializer=bi)(fc_advantage)
        fc_value = Dense(512, use_bias=False, kernel_initializer=wei, activation='relu')(h_flat)
        value = Dense(1, kernel_initializer=wei, use_bias=False, bias_initializer=bi)(fc_value)
        z = Lambda(lambda x: x[1] + x[0] - K.mean(advantage, axis=1, keepdims=True), output_shape=(NUM_ACTIONS,))([advantage, value])
        model = Model(inputs=X, outputs=z)
        model.trainable = trainable
        return(model, z)

    #Update Learning rate
    def lr_up(self):
        self.learning_rate=self.learning_rate * (0.99 ** (self.decay_step/100))
        self.decay_step+=1
        K.set_value(self.model_train.optimizer.lr, self.learning_rate)

    def tn_build():
        Y=Input(shape=SHAPE[1:], dtype='float32')
        model,Q=Q_Agent.infer(Y, trainable=False, init=initializers.zeros())
        return(model)

    #update epsilon
    def eps_up(self):
        self.epsilon=np.maximum(EPSILON_FINAL,self.epsilon - (EPSILON_INITIAL - EPSILON_FINAL) / EXPLORATION_STEPS)
     
    #Train
    def train(self):
        q_batch,state_now,mask_b=self.sbat()
        h=self.model_train.fit([state_now,mask_b],np.sum(mask_b*q_batch, axis=1),verbose=0)
        return(h.history['loss'][0])
    
    #Restore
    def res(self):
        self.model_target.load_weights(os.path.join('gdrive/My Drive/Colab Notebooks/', 'model_duel_target.h5'))
        episode, decay_step, eps, learning_rate=np.load(os.path.join('gdrive/My Drive/Colab Notebooks/', 'params_duel.npy'))
        self.model_train.load_weights(os.path.join('gdrive/My Drive/Colab Notebooks/', 'model_duel_train.h5'))
        return(eps, int(episode), learning_rate, int(decay_step))
         
    #update target model
    def t_net_up(self):
        self.model_target.set_weights(self.model_train.get_weights())

In [15]:
if __name__ == '__main__':
    env = wrap_env(gym.make('BreakoutDeterministic-v4'))
    ROOT = 'Model_DQN'
    ag = Q_Agent(ROOT)
    is_done = False
    cnt_frames = 0
    i=0
    j=0
    while (i<1):
        frame = env.reset()
        is_done = False
        frame = down_sample(rgb2gray(frame))
        frame_stack = [frame, frame, frame, frame]
        points = 0
        while not is_done:
            #env.render()
            state_current = frame_stack[-STATE_LENGTH:]
            action, _ = ag.predict(np.expand_dims(np.transpose(state_current, [1, 2, 0]), axis=0))
            eps = ag.epsilon
            if np.random.random() < eps:
                action = env.action_space.sample()
            frame, reward, is_done, _ = env.step(action)
            points += reward
            j=points
            frame_next = down_sample(rgb2gray(frame))
            frame_stack.append(frame_next)
            frame_stack = frame_stack[-STATE_LENGTH:]
        print('Scores: %d' % points)
        i+=1
    env.close()
    show_video()

Scores: 274
